# Batch: simple vs RL tabular MM

Runs `mm_compare` for both `simple` and `rl_tabular` market makers over multiple seeds, then plots MTM PnL averages (with/without std bands) and summary stats.

In [ ]:
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', None)

# ---- parameters ----
run_simulations = True  # set False to skip running and just analyze existing logs
num_runs = 5           # seeds to run
base_seed = 12345      # RNG seed to generate seeds list
ticker = 'AAPL'
historical_date = '20000101'
start_time = '00:00:01'
end_time = '23:59:00'
mm_types = ['simple', 'rl_tabular']
log_root = Path('..') / 'log'
# --------------------

rng = np.random.default_rng(base_seed)
seeds = rng.integers(low=1, high=2**31-1, size=num_runs)
seeds

## Run simulations

In [ ]:
log_root.mkdir(exist_ok=True)
runs = []
if run_simulations:
    for mm in mm_types:
        for seed in seeds:
            run_name = f"{mm}_seed{seed}"
            cmd = [
                'python', 'abides.py',
                '-c', 'mm_compare',
                '-t', ticker,
                '-d', historical_date,
                '--start-time', start_time,
                '--end-time', end_time,
                '--seed', str(seed),
                '--log_dir', run_name,
                '--mm-type', mm,

            ]
            print('Running:', ' '.join(cmd))
            result = subprocess.run(cmd, cwd=Path('..'))
            runs.append({'mm': mm, 'seed': seed, 'log_dir': log_root / run_name, 'returncode': result.returncode})
else:
    runs = [{'mm': mm, 'seed': seed, 'log_dir': log_root / f"{mm}_seed{seed}", 'returncode': 0}
            for mm in mm_types for seed in seeds]
runs

## Load MTM series

In [ ]:
def load_mtm_series(log_dir, pattern):
    files = list(log_dir.glob(pattern))
    if not files:
        return None
    df = pd.read_pickle(files[0])
    states = df[df['EventType'] == 'STATE']
    if states.empty:
        return None
    s = pd.DataFrame(list(states['Event']))
    s = s.set_index('time')
    s.index = pd.to_datetime(s.index)
    return s['mtm'].sort_index()

mtm_series = {mm: [] for mm in mm_types}
for run in runs:
    if run['returncode'] != 0:
        print('Skipping failed run', run)
        continue
    mm = run['mm']
    pattern = 'RL_TABULAR_MARKET_MAKER_AGENT_*.bz2' if mm == 'rl_tabular' else 'MARKET_MAKER_AGENT_*.bz2'
    series = load_mtm_series(run['log_dir'], pattern)
    if series is not None:
        mtm_series[mm].append({'seed': run['seed'], 'series': series})

mtm_series

## MTM mean/std over time (aligned)

In [ ]:
def align_and_stats(series_list, freq='5T'):
    if not series_list:
        return None, None
    aligned = []
    for item in series_list:
        res = item['series'].resample(freq).last()
        aligned.append(res)
    df = pd.concat(aligned, axis=1)
    mean = df.mean(axis=1)
    std = df.std(axis=1)
    return mean, std

stats_over_time = {}
for mm in mm_types:
    mean, std = align_and_stats(mtm_series[mm])
    stats_over_time[mm] = {'mean': mean, 'std': std}

plt.figure(figsize=(10,4))
for mm in mm_types:
    data = stats_over_time[mm]
    if data['mean'] is not None:
        data['mean'].plot(label=f"{mm} mean")
plt.title('MTM mean over time (5 min)')
+plt.legend()
plt.tight_layout()

plt.figure(figsize=(10,4))
for mm in mm_types:
    data = stats_over_time[mm]
    if data['mean'] is not None:
        m = data['mean']
        s = data['std']
        plt.plot(m.index, m, label=f"{mm} mean")
        plt.fill_between(m.index, m - s, m + s, alpha=0.2, label=f"{mm} ±1 std")
plt.title('MTM mean ± std over time (5 min)')
plt.legend()
plt.tight_layout()

## Summary tables (final PnL)

In [ ]:
def final_pnl(series):
    if series is None or series.empty:
        return None
    return series.iloc[-1] - series.iloc[0]

records = []
for mm in mm_types:
    for item in mtm_series[mm]:
        pnl = final_pnl(item['series'])
        if pnl is not None:
            records.append({'mm': mm, 'seed': item['seed'], 'pnl': pnl})

pnl_df = pd.DataFrame(records)
pnl_df.head()


In [ ]:
if not pnl_df.empty:
    summary = pnl_df.groupby('mm')['pnl'].agg(['mean', 'std', 'median', 'min', 'max', 'count'])
    display(summary)
else:
    print('No PnL data to summarize')